In [ ]:
import cartopy.crs as ccrs
from datetime import datetime
import pylab as plt
import pandas as pd
import json
import gc
from glob import glob
import numpy as np
import pytz
from spacepy import pycdf
import lib_search_dispersion
%matplotlib inline

In [ ]:
dfs = []

for output in glob('data/Long_Term_Trend_F*.csv'):
    dfc = pd.read_csv(output, parse_dates=['start_time', 'end_time'])
    dfs.append(dfc)

df_all = pd.concat(dfs).reset_index()
df_all

In [ ]:
df_kp = pd.read_csv('/home/dedasilv/disp/data/Kp_ap_since_1932.txt', comment='#', sep='\\s+',
                    names='year month day hour hour2 days days_m Kp ap D'.split())

df_kp.insert(0, 'timestamp', [
    datetime(
        int(row.year), int(row.month), int(row.day), int(row.hour),
        tzinfo=pytz.utc
    )
    for (_, row) in df_kp.iterrows()
])

In [ ]:
df_all['Kp'] = [
    df_kp.iloc[df_kp.timestamp.searchsorted(row.start_time)].Kp
    for (_, row)
    in df_all.iterrows()
]
df_all.head()

In [ ]:
#df = df_all[df_all.Kp >= 6].copy()
df = df_all.copy()

In [ ]:
fnames = []

for cf in glob("case_files/Long*.json"):
    fnames.extend(json.load(open(cf))['DMSP_FLUX_FILES'])

all_mlat = []
all_mlt = []

for i, f in enumerate(fnames):
    print(f'{i} out of {len(fnames)}')
    try:
        cdf = pycdf.CDF(f)
        all_mlat.append(cdf['SC_AACGM_LAT'][:].astype(np.float32))                                                                 
        all_mlt.append(cdf['SC_AACGM_LTIME'][:].astype(np.float32))
        cdf.close()
    except pycdf.CDFError:
        pass

In [ ]:
gc.collect()
all_mlat = np.concatenate(all_mlat)
gc.collect()
all_mlt = np.concatenate(all_mlt)
all_mlt *= 2 * np.pi / 24.0
gc.collect()

In [ ]:
dmsp_files = {}

for i, row in df_all.iterrows():
    print(f'{i} out of {len(df_all.index)}')
    try:
        cdf = pycdf.CDF(row.file)
        res = {}
        res['t'] = np.array([t.replace(tzinfo=pytz.utc) for t in cdf['Epoch'][:]])
        res['mlat'] = cdf['SC_AACGM_LAT'][:].astype(np.float32)               
        res['mlt'] = 2 * np.pi * (1/24.) *cdf['SC_AACGM_LTIME'][:].astype(np.float32)
        cdf.close()
        dmsp_files[row.file] = res
    except pycdf.CDFError:
        pass

In [ ]:
mlat = []
mlt = []
for _, row in df.iterrows():
    idx = dmsp_files[row.file]['t'].searchsorted(row.start_time)
    mlat.append(dmsp_files[row.file]['mlat'][idx])
    mlt.append(dmsp_files[row.file]['mlt'][idx])
df['mlat'] = mlat
df['mlt'] = mlt
df

In [ ]:
def plot_points(df_masked, pole):
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(8, 8))
    ax.plot(df_masked.mlt, df_masked.mlat, 'ro')
    ax.grid(True)
    if pole=='north':
        ax.set_rlim(90, 60)
    else:
        ax.set_rlim(-90, -60)
    ax.set_theta_offset(-np.pi/2)
    ax.set_title(f'Viewed from {pole.capitalize()} Magnetic Pole\nDispersion Events 2010-2014', fontweight='bold')

In [ ]:
plot_points(df[df.mlat > 0], 'north')
plot_points(df[df.mlat < 0], 'south')

In [ ]:
def plot_coverage(mlt, mlat, pole):
    mlt_bins = np.linspace(0, 2*np.pi * 1.01, 24*4)
    
    if pole == 'north':
        mlat_bins = np.linspace(45, 90.01, 30)
    else:
        mlat_bins = np.linspace(-90, -45, 30)
    
    dwell, xedges, yedges = np.histogram2d(mlt, mlat, bins=[mlt_bins, mlat_bins])
    dwell /= 3600

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(12, 8))
    im = ax.pcolor(xedges[:-1], yedges[:-1], dwell.T, cmap='jet')
    ax.grid(True)
    
    if pole=='north':
        ax.set_rlim(90, 60)
    else:
        ax.set_rlim(-90, -60)
    
    ax.set_theta_offset(-np.pi/2)
    ax.set_title(f'Viewed from {pole.capitalize()} Magnetic Pole\nDwell Time F16/F17/F18 2010-2014', fontweight='bold')
    ax.set_facecolor('white')
    bbox = dict(boxstyle="round", ec="black", fc="white")
    plt.setp(ax.get_yticklabels(), bbox=bbox)
    plt.colorbar(im).set_label('Dwell Time per Bin (Hours)', fontsize=20)

In [ ]:
gc.collect()
plot_coverage(all_mlt[all_mlat > 0], all_mlat[all_mlat > 0], 'north')

In [ ]:
gc.collect()
plot_coverage(all_mlt[all_mlat < 0], all_mlat[all_mlat < 0], 'south')

In [ ]:
def plot_normalized_event_rate(df_masked, mlt, mlat, pole):
    mlt_bins = np.linspace(0, 2*np.pi * 1.01, 24*4)

    if pole == 'north':
        mlat_bins = np.linspace(45, 90.01, 30)
    else:
        mlat_bins = np.linspace(-90, -45, 30)

    H, xedges, yedges = np.histogram2d(df_masked.mlt, df_masked.mlat, bins=[mlt_bins, mlat_bins])
    Hnorm, _, _ = np.histogram2d(mlt, mlat, bins=[mlt_bins, mlat_bins])

    scaled = H.copy()
    #scaled[Hnorm < 0.15 * Hnorm.max()] = 0
    scaled[Hnorm>0] /= Hnorm[Hnorm>0]
    scaled /= scaled.max()

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(12, 8))
    im = ax.pcolor(xedges[:-1], yedges[:-1], scaled.T, cmap='jet')
    ax.grid(True)

    if pole=='north':
        ax.set_rlim(90, 60)
    else:
        ax.set_rlim(-90, -60)
    
    ax.set_theta_offset(-np.pi/2)
    ax.set_title(f'Viewed from {pole.capitalize()} Magnetic Pole\nDispersion Events 2010-2014', fontweight='bold')
    ax.set_facecolor('white')
    bbox = dict(boxstyle="round", ec="black", fc="white")
    plt.setp(ax.get_yticklabels(), bbox=bbox)
    plt.colorbar(im).set_label('Event Rate Rate Normalized by Dwell Time', fontsize=20)
    None

In [ ]:
gc.collect()
plot_normalized_event_rate(df[df.mlat > 0], all_mlt[all_mlat > 0], all_mlat[all_mlat > 0], 'north')

In [ ]:
gc.collect()
plot_normalized_event_rate(df[df.mlat < 0], all_mlt[all_mlat < 0], all_mlat[all_mlat < 0], 'south')

In [ ]:
df_masked = df[df.mlat > 0]
bins = np.arange(0, 24)
plt.figure(figsize=(12, 4))

plt.subplot(121)
plt.hist(24 / (2. * np.pi ) * all_mlt[all_mlat < 0], bins=bins)
plt.title('Magnetic Northern Hemisphere\nOrbital Visits F16/F17/F18', fontweight='bold')
plt.ylabel('Bin Count')
plt.grid(True, linestyle='dashed', color='gray')
plt.xlabel('MLT')
plt.subplot(122)
plt.hist(24 / (2. * np.pi ) * df_masked.mlt, bins=bins)
plt.title('Magnetic Northern Hemisphere\nEvent Locations F16/F17/F18', fontweight='bold')
plt.grid(True, linestyle='dashed', color='gray')
plt.xlabel('MLT')

In [ ]:
df_masked = df[df.mlat < 0]
bins = np.arange(0, 24)
plt.figure(figsize=(12, 4))

plt.subplot(121)
plt.hist(24 / (2. * np.pi ) * all_mlt[all_mlat < 0], bins=bins)
plt.title('Magnetic Southern Hemisphere\nOrbital Visits F16/F17/F18', fontweight='bold')
plt.ylabel('Bin Count')
plt.grid(True, linestyle='dashed', color='gray')
plt.xlabel('MLT')

plt.subplot(122)
plt.hist(24 / (2. * np.pi ) * df_masked.mlt, bins=bins)
plt.title('Magnetic Southern Hemisphere\nEvent Locations F16/F17/F18', fontweight='bold')
plt.grid(True, linestyle='dashed', color='gray')
plt.xlabel('MLT')

df.to_csv('df_out.csv', index=0)
df[df.mlat <0].to_csv('df_south.csv', index=0)


import os, shutil

for t in df[df.mlat <0].start_time:
    plots = glob(f'plots/**/*{t.isoformat()}*.png', recursive=True)
    for plot in plots:
        shutil.copy(plot, os.path.join('/home/dedasilv/south/', os.path.basename(plot)))

for t in df[df.mlat >0].start_time:
    plots = glob(f'plots/**/*{t.isoformat()}*.png', recursive=True)    
    for plot in plots:
        shutil.copy(plot, os.path.join('/home/dedasilv/north/', os.path.basename(plot)))